In [13]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob
import seaborn as sns
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import mannwhitneyu
from dotenv import load_dotenv

load_dotenv('./Credentials.env',override=True)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =str(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))
os.environ['GCLOUD_PROJECT'] = str(os.getenv("GCLOUD_PROJECT"))

%load_ext google.cloud.bigquery
from google.cloud import bigquery
client=bigquery.Client()
from google.cloud import bigquery_storage_v1

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [15]:
%%bigquery Train_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2015 and order_year<2022

Executing query with job ID: 5a37f2d3-202e-4014-a445-0adfb38a00b4
Query executing: 0.38s


ERROR:
 404 Not found: Table som-nero-phi-jonc101:blood_culture_stewardship.note_cohort was not found in location US; reason: notFound, message: Not found: Table som-nero-phi-jonc101:blood_culture_stewardship.note_cohort was not found in location US

Location: US
Job ID: 5a37f2d3-202e-4014-a445-0adfb38a00b4



In [11]:
Train_set_df[(Train_set_df.positive_blood_culture_in_week==1)|(Train_set_df.positive_blood_culture==1)][['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

(8895, 3)

In [72]:
%%bigquery Val_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2022 and order_year<2023

Query is running:   0%|          |

Downloading:   0%|          |

In [73]:
%%bigquery Test_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2023 

Query is running:   0%|          |

Downloading:   0%|          |

In [74]:
missing_percentage = Test_set_df.isnull().mean() * 100
missing_percentage[missing_percentage>80]

min_neutrophils             86.818038
max_neutrophils             86.818038
avg_neutrophils             86.818038
median_neutrophils          86.818038
min_lymphocytes             86.601230
max_lymphocytes             86.601230
avg_lymphocytes             86.601230
median_lymphocytes          86.601230
min_procalcitonin           91.520814
max_procalcitonin           91.520814
avg_procalcitonin           91.520814
median_procalcitonin        91.520814
bacteremia                  87.760170
infective_endocarditis      98.557237
septic_thrombophlebitis     99.428414
vascular_graft_infection    98.415326
CRBSI                       98.707032
infectious_discitis         99.802901
epidural_abscess            92.786187
septic_arthritis            98.545412
meningitis                  99.105172
meningitis_bacteria         99.854147
cholangitis                 97.780669
dtype: float64

In [75]:
Train_set_df['ed_arrival_datetime'] = pd.to_datetime(Train_set_df['ed_arrival_datetime'])
Train_set_df['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df['blood_culture_order_datetime'])
Train_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Train_set_df['earliest_iv_antibiotic_datetime'])

Val_set_df['ed_arrival_datetime'] = pd.to_datetime(Val_set_df['ed_arrival_datetime'])
Val_set_df['blood_culture_order_datetime'] = pd.to_datetime(Val_set_df['blood_culture_order_datetime'])
Val_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Val_set_df['earliest_iv_antibiotic_datetime'])


Test_set_df['ed_arrival_datetime'] = pd.to_datetime(Test_set_df['ed_arrival_datetime'])
Test_set_df['blood_culture_order_datetime'] = pd.to_datetime(Test_set_df['blood_culture_order_datetime'])
Test_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Test_set_df['earliest_iv_antibiotic_datetime'])


# Calculate the difference in hours:
Train_set_df['hours_between_ed_cult'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['ed_arrival_datetime']).dt.total_seconds() / 3600
Test_set_df['hours_between_ed_cult'] = (Test_set_df['blood_culture_order_datetime'] - Test_set_df['ed_arrival_datetime']).dt.total_seconds() / 3600
Val_set_df['hours_between_ed_cult'] = (Val_set_df['blood_culture_order_datetime'] - Val_set_df['ed_arrival_datetime']).dt.total_seconds() / 3600


Train_set_df['hours_between_cult_abx'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600
Test_set_df['hours_between_cult_abx'] = (Test_set_df['blood_culture_order_datetime'] - Test_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600
Val_set_df['hours_between_cult_abx'] = (Val_set_df['blood_culture_order_datetime'] - Val_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600


In [76]:
Identifiers=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']
Labels=['positive_blood_culture','positive_blood_culture_in_week']
Labs=['min_heartrate','max_heartrate', 'avg_heartrate', 'median_heartrate',
       'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
       'min_temp', 'max_temp', 'avg_temp', 'median_temp', 'min_sysbp',
       'max_sysbp', 'avg_sysbp', 'median_sysbp', 'min_diasbp',
       'max_diasbp', 'avg_diasbp', 'median_diasbp', 'min_wbc', 'max_wbc',
       'avg_wbc', 'median_wbc', 'min_neutrophils', 'max_neutrophils',
       'avg_neutrophils', 'median_neutrophils', 'min_lymphocytes',
       'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes',
       'min_hgb', 'max_hgb', 'avg_hgb', 'median_hgb', 'min_plt',
       'max_plt', 'avg_plt', 'median_plt', 'min_na', 'max_na', 'avg_na',
       'median_na', 'min_hco3', 'max_hco3', 'avg_hco3', 'median_hco3',
       'min_bun', 'max_bun', 'avg_bun', 'median_bun', 'min_cr', 'max_cr',
       'avg_cr', 'median_cr', 'min_lactate', 'max_lactate', 'avg_lactate',
       'median_lactate', 'min_procalcitonin', 'max_procalcitonin',
       'avg_procalcitonin', 'median_procalcitonin']
Demos=[ 'gender','age']
ABX=['vanc', 'zosyn', 'vanc_zosyn', 'other_ABX']
Time_Varient_features=['hours_between_ed_cult', 'hours_between_cult_abx']
Diagnosis= ['bacteremia', 'septic_shock', 'infective_endocarditis',
       'septic_thrombophlebitis', 'vascular_graft_infection', 'CRBSI',
       'infectious_discitis', 'epidural_abscess', 'septic_arthritis',
       'meningitis', 'meningitis_bacteria', 'cholangitis',
       'bacterial_cholangitis', 'pyelonephritis',
       'acute_bacterial_pyelonephritis', 'severe_pneumonia',
       'acute_hematogenous_osteomyelitis', 'asplenia',
       'immunocompromised_state', 'severe_cellulitis', 'cystitis',
       'prostatitis', 'CAP', 'diabetic_foot_infection', 'colitis',
       'aspiration_pneumonia', 'uncomplicated_cholecystitis',
       'uncomplicated_diverticulitis', 'Uncomplicated_pancreatitis']

Feature_set=Identifiers+Labels+Labs+Demos+ABX+Diagnosis+Time_Varient_features (# select Features based on experiment)
Train_set_df=Train_set_df[Feature_set]
    
Test_set_df=Test_set_df[Feature_set]

Val_set_df=Val_set_df[Feature_set]
                       

In [77]:
Train_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)
Val_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)
Test_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)


In [78]:
Train_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1
Test_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1
Val_set_df['datapoint'] = Val_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1


In [79]:
Train_set_df['gender'] = Train_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
Test_set_df['gender'] = Test_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
Val_set_df['gender'] = Val_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))


In [80]:
Train_set_df.shape[0]+Test_set_df.shape[0]+Val_set_df.shape[0]

135483

# Logistic Regression

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from fancyimpute import IterativeImputer as FancyIterativeImputer

In [15]:
Train_set_df['Label']= (Train_set_df['positive_blood_culture']|Train_set_df['positive_blood_culture_in_week'])
X_train = Train_set_df.drop(columns=['positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','Label'])
y_train = Train_set_df['Label']

# Prepare the test data
Test_set_df['Label']= (Test_set_df['positive_blood_culture']|Test_set_df['positive_blood_culture_in_week'])
X_test = Test_set_df.drop(columns=['positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'])
y_test = Test_set_df['Label']


Val_set_df['Label']= (Val_set_df['positive_blood_culture']|Val_set_df['positive_blood_culture_in_week'])
X_val = Val_set_df.drop(columns=['positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'])
y_val = Val_set_df['Label']

## Fancy_Imputer

In [291]:
fancy_imputer = FancyIterativeImputer(max_iter=10, random_state=0)
fancy_imputer.fit(X_train)

X_train = pd.DataFrame(fancy_imputer.transform(X_train), columns=X_train.columns)

# Impute the missing values in the test set
X_test = pd.DataFrame(fancy_imputer.transform(X_test), columns=X_train_im.columns)

# Impute the missing values in the validation set
X_val = pd.DataFrame(fancy_imputer.transform(X_val), columns=X_train_im.columns)


## Median Imputer

In [16]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
imputer.fit(X_train)  # Fit on the training data to calculate medians
# Step 2: Impute the missing values in all datasets
X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


## For study with including Fever as Features

In [ ]:

X_train_im['Fever'] = X_train_im['max_temp'].apply(lambda x: 1 if x > 100.4 else 0)
X_val['Fever'] = X_val['max_temp'].apply(lambda x: 1 if x > 100.4 else 0)
X_test['Fever'] = X_test['max_temp'].apply(lambda x: 1 if x > 100.4 else 0)


## Scale paramters

In [20]:
from sklearn.preprocessing import StandardScaler
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Transform the training, test, and validation data
X_train2 = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test2 = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)
X_val2 = pd.DataFrame(scaler.transform(X_val), columns=X_train.columns)

## Model Training

## Search over paramters

In [ ]:
ratio = np.sum(y_train == 0) / np.sum(y_train == 1)

custom_weights = [
    None,                  
    "balanced",                  
    {0: 1, 1: 2},                
    {0: 1, 1: 3},                
    {0: 1, 1: 5},
    {0: 1, 1: ratio} 
]
param_grid = [
    {
        "penalty": ["l1"],
        "solver": ["liblinear", "saga"],
        "C": [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
        "class_weight": custom_weights,
        "max_iter": [1000]
    },
    {
        "penalty": ["l2"],
        "solver": ["liblinear", "saga", "lbfgs", "sag"],
        "C": [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
        "class_weight": custom_weights,
        "max_iter": [1000]
    }
]

In [ ]:
# Concatenate train and val so GridSearchCV 
X_search = np.vstack([X_train2, X_val2])
y_search = np.concatenate([y_train, y_val])

test_fold = np.array([-1] * len(y_train) + [0] * len(y_val))
ps = PredefinedSplit(test_fold)

In [ ]:
base_model = LogisticRegression()

grid = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=ps,               
    refit=True,         
    n_jobs=-1,
    verbose=1
)

grid.fit(X_search, y_search)

## LR model

In [ ]:
best_model = grid.best_estimator_
y_pred_prob_test = best_model.predict_proba(X_test2)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob_test)

In [21]:
best_params = grid.best_params_.copy()
model_l2 = LogisticRegression(**best_params)
model_l2.fit(X_train2, y_train)

# Predict probabilities for the test set
y_pred_prob_l2 = model_l2.predict_proba(X_test2)[:, 1]

# Calculate the AUC
auc_l2 = roc_auc_score(y_test, y_pred_prob_l2)
print(f"AUC for the test set with L1 regularization: {auc_l2:.4f}")

AUC for the test set with L1 regularization: 0.7152


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
import numpy as np


# L2 Regularization (Ridge)
model_l2 = LogisticRegression(max_iter=2000,class_weight='balanced')#, C=0.01, penalty='l1')#solver='lbfgs'
model_l2.fit(X_train2, y_train)

# Predict probabilities for the test set
y_pred_prob_l2 = model_l2.predict_proba(X_test2)[:, 1]

# Calculate the AUC
auc_l2 = roc_auc_score(y_test, y_pred_prob_l2)
print(f"AUC for the test set with L2 regularization: {auc_l2:.4f}")


# Calculate the AUC
yt_pred_prob_l2 = model_l2.predict_proba(X_val2)[:, 1]
auc_l2 = roc_auc_score(y_val, yt_pred_prob_l2)
print(f"AUC for the validation set with L2 regularization: {auc_l2:.4f}")

In [23]:
import numpy as np
from sklearn.metrics import roc_curve, confusion_matrix

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_l2)

# Find the threshold where sensitivity (TPR) >= 0.9
threshold_index = np.argmax(tpr >= 0.95)
optimal_threshold = thresholds[threshold_index]

# Use the optimal threshold to make binary predictions
y_pred_optimal = (y_pred_prob_l2 >= optimal_threshold).astype(int)

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_optimal).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)  # Sensitivity or Recall
specificity = tn / (tn + fp)  # Specificity
ppv = tp / (tp + fp)          # Positive Predictive Value (Precision)
npv = tn / (tn + fn)          # Negative Predictive Value

print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"PPV (Precision): {ppv:.4f}")
print(f"NPV: {npv:.4f}")

Optimal Threshold: 0.2960
Sensitivity: 0.9503
Specificity: 0.1668
PPV (Precision): 0.0325
NPV: 0.9913


In [26]:
# save model
import pickle

# Assuming model_l2 is your trained Logistic Regression model
with open('logistic_regression_modelI.pkl', 'wb') as model_file:
    pickle.dump(model_l2, model_file)

In [27]:
# Get the coefficients
coefficients = model_l2.coef_[0]  # Coefficients for the features
intercept = model_l2.intercept_[0]  # Intercept term

# Print the coefficients and intercept
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [ 0.20057557  0.00179579  0.33366419 -0.207842    0.19397862 -0.09343402
 -0.08960952 -0.21265723  0.1118055   0.09470426  0.02193637  0.20037391]
Intercept: -0.21577695703453317


In [28]:
coefficients = model_l2.coef_[0] 
# Get the feature names from your training data (assuming X_train2 is a DataFrame)
feature_names = X_train2.columns

# Identify non-zero coefficients and get corresponding feature names
non_zero_indices = np.where(coefficients != 0)[0]
non_zero_features = feature_names[non_zero_indices]
non_zero_coefficients = coefficients[non_zero_indices]

# Print the features and their corresponding non-zero coefficients
for feature, coef in zip(non_zero_features, non_zero_coefficients):
    print(f"Feature: {feature}, Coefficient: {coef}")

Feature: max_heartrate, Coefficient: 0.20057556921051362
Feature: max_resprate, Coefficient: 0.0017957877100702754
Feature: Fever, Coefficient: 0.33366419300147093
Feature: min_sysbp, Coefficient: -0.20784200224952826
Feature: max_wbc, Coefficient: 0.19397862134956015
Feature: min_na, Coefficient: -0.09343402422707259
Feature: min_hco3, Coefficient: -0.08960951907624168
Feature: min_plt, Coefficient: -0.21265723295675015
Feature: max_cr, Coefficient: 0.11180549512835596
Feature: max_lactate, Coefficient: 0.09470426443307634
Feature: gender, Coefficient: 0.021936369224784914
Feature: age, Coefficient: 0.20037390877239228
